In [55]:
from collections import namedtuple
import tensorflow as tf


In [75]:

from deepClassifier.constants import *
from deepClassifier.utils import read_yaml, create_directories
import urllib.request as request
from pathlib import Path




In [76]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    params_image_size: list
    params_learning_rate: float
    params_include_top: bool
    params_weights: str
    params_classes: int

In [77]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            params_image_size=self.params.IMAGE_SIZE,
            params_learning_rate=self.params.LEARNING_RATE,
            params_include_top=self.params.INCLUDE_TOP,
            params_weights=self.params.WEIGHTS,
            params_classes=self.params.CLASSES
        )

        return prepare_base_model_config


In [88]:
# VGG16 Visual Geometry Group from Oxford
class PrepareBaseModel:
    def __init__(self, config:PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.vgg16.VGG16(
            input_shape = self.config.params_image_size,
            weights = self.config.params_weights,
            include_top = self.config.params_include_top#drop ann and keep cnn 
        ) #16 number of layers

        self.save_model(path=self.config.base_model_path, model=self.model)

    @staticmethod
    def _prepare_full_model(model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in model.layers:
                model.trainable = False
        elif (freeze_till is not None) and (freeze_till> 0):
            for layer in model.layers[:-freeze_till]:
                model.trainable = False
        flatten_in = tf.keras.layers.Flatten()(model.output)#function approach
        prediction = tf.keras.layers.Dense(
            units = classes,
            activation = "softmax"
        )(flatten_in)#functional approach

        full_model = tf.keras.models.Model(
            inputs = model.output,
            outputs = prediction
        )
        full_model.compile(
            optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )
        full_model.summary()
        return full_model
#freeze_all, freeze_till experiment try and error
    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            model=self.model,
            classes=self.config.params_classes,
            freeze_all=True,
            freeze_till=None,
            learning_rate=self.config.params_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)

    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)



In [89]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
    prepare_base_model.update_base_model()
except Exception as e:
    raise e

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 7, 7, 512)]       0         
                                                                 
 flatten_2 (Flatten)         (None, 25088)             0         
                                                                 
 dense_1 (Dense)             (None, 2)                 50178     
                                                                 
Total params: 50,178
Trainable params: 50,178
Non-trainable params: 0
_________________________________________________________________


In [52]:
os.chdir("/Users/syedaskari/MSDC!/Deep Learing/Deep-Learing/")

In [61]:
os.getcwd()

'/Users/syedaskari/MSDC!/Deep Learing/Deep-Learing'

In [54]:
os.listdir()

['research',
 'artifacts',
 'init_setup.sh',
 '.DS_Store',
 'requirements.txt',
 '.pytest_cache',
 '.tox',
 'pyproject.toml',
 'params.yaml',
 'tests',
 'README.md',
 'setup.py',
 'CNN',
 'dvc.yaml',
 'logs',
 '.mypy_cache',
 '.gitignore',
 'configs',
 '.github',
 'tox.ini',
 'template.py',
 'setup.cfg',
 'venv',
 'CNN_Classfier',
 '.git',
 '.vscode',
 'LabAssignment',
 'LICENCE',
 'requirements_dev.txt',
 'src']